In [5]:
from python_workflow_definition.aiida import write_workflow_json, construct_wg_qe
from python_workflow_definition.jobflow import load_workflow_json
from jobflow.managers.local import run_locally

from aiida import load_profile
load_profile()

workflow_json_filename =  "workflow_qe_aiida.json"

In [6]:
from quantum_espresso_workflow import (
    generate_structures,
    get_bulk_structure,
    calculate_qe,
    plot_energy_volume_curve,
)

from python_workflow_definition.aiida import pickle_node, construct_wg_qe
from python_workflow_definition.shared import get_list, get_dict

# from python_workflow_definition.pyiron_base import  get_dict
from aiida_workgraph import WorkGraph, task
from typing import Any

load_profile()

get_list_task = task.pythonjob(get_list)
get_dict_task = task.pythonjob(get_dict)
pickle_node_task = task.pythonjob(pickle_node)
get_bulk_structure_task = task.pythonjob()(get_bulk_structure)
generate_structures_task = task.pythonjob()(generate_structures)
calculate_qe_task = task.pythonjob(outputs=["energy", "volume", "structure"])(
    calculate_qe
)
plot_energy_volume_curve_task = task.pythonjob()(plot_energy_volume_curve)

strain_lst = [0.9, 0.95, 1.0, 1.05, 1.1]

wg = construct_wg_qe(
    get_dict_task=get_dict_task,
    get_list_task=get_list_task,
    pickle_node_task=pickle_node_task,
    get_bulk_structure_task=get_bulk_structure_task,
    calculate_qe_task=calculate_qe_task,
    generate_structures_task=generate_structures_task,
    plot_energy_volume_curve_task=plot_energy_volume_curve_task,
    strain_lst=strain_lst
)

wg
wg.to_html("wg-aiida-fix.html")

In [7]:
_ = write_workflow_json(wg=wg, file_name=workflow_json_filename)


In [8]:

flow = load_workflow_json(file_name=workflow_json_filename)

KeyError: 0

In [ ]:
result = run_locally(flow)
result

2025-03-15 11:21:45,872 INFO Started executing jobs locally
2025-03-15 11:21:46,028 INFO Starting job - add_x_and_y (31bed0f7-49cd-4e63-9a53-8c9679e74dd2)
2025-03-15 11:21:46,030 INFO Finished job - add_x_and_y (31bed0f7-49cd-4e63-9a53-8c9679e74dd2)
2025-03-15 11:21:46,031 INFO Starting job - add_x_and_y_and_z (7d74164e-3d5b-40d0-b72b-4fc5434a98e1)
2025-03-15 11:21:46,033 INFO Finished job - add_x_and_y_and_z (7d74164e-3d5b-40d0-b72b-4fc5434a98e1)
2025-03-15 11:21:46,033 INFO Finished executing jobs locally


{'31bed0f7-49cd-4e63-9a53-8c9679e74dd2': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))},
 '7d74164e-3d5b-40d0-b72b-4fc5434a98e1': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))}}